<a href="https://colab.research.google.com/github/anirudhc5/ACLHacks2024/blob/main/AcadmiesHacks2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("cyberbullying_tweets.csv")
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [4]:
df["classification"] = df["cyberbullying_type"].replace({"not_cyberbullying":0, "cyberbullying":1, "other_cyberbullying":1, "age":1, "ethnicity":1, "gender":1, "religion":1,})
df = df.drop("cyberbullying_type", axis=1)
df.head()

,tweet_text,classification
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0
2,@XochitlSuckkks a classy whore? Or more red ve...,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0
4,@RudhoeEnglish This is an ISIS account pretend...,0


In [5]:
# Load dataset
df = pd.read_csv('cyberbullying_tweets.csv')
df["classification"] = df["cyberbullying_type"].replace({"not_cyberbullying":0, "cyberbullying":1, "other_cyberbullying":1, "age":1, "ethnicity":1, "gender":1, "religion":1,})
df = df.drop("cyberbullying_type", axis=1)
df.head()



,tweet_text,classification
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0
2,@XochitlSuckkks a classy whore? Or more red ve...,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0
4,@RudhoeEnglish This is an ISIS account pretend...,0


In [6]:
texts = df["tweet_text"].tolist()
labels = df["classification"].tolist()

In [8]:
!pip install huggingface_hub

In [9]:

# Load model and tokenizer
model_name = "openai-community/roberta-base-openai-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)


In [10]:
def map_example_to_dict(input_ids, attention_masks, labels):
    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
    }, labels

# Prepare dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    encodings['input_ids'],
    encodings['attention_mask'],
    labels
))
train_dataset = train_dataset.map(map_example_to_dict).shuffle(1000).batch(16)



In [11]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, BatchNormalization
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from transformers import create_optimizer

In [16]:
from tensorflow import keras

In [13]:
optimizer, lr_scheduler = create_optimizer(
    init_lr = 5e-5,
    num_train_steps = len(train_dataset) * 5,
    num_warmup_steps=0
)

In [17]:
model.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy("accuracy")]
)

In [22]:
model.save_pretrained("/content/model")
tokenizer.save_pretrained("/content/model")

('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.json',
 '/content/model/merges.txt',
 '/content/model/added_tokens.json',
 '/content/model/tokenizer.json')

In [25]:
!zip -r /content/model.zip /content/model

  adding: content/model/ (stored 0%)
  adding: content/model/vocab.json (deflated 59%)
  adding: content/model/merges.txt (deflated 53%)
  adding: content/model/tokenizer_config.json (deflated 76%)
  adding: content/model/tf_model.h5 (deflated 7%)
  adding: content/model/special_tokens_map.json (deflated 52%)
  adding: content/model/config.json (deflated 50%)
  adding: content/model/tokenizer.json (deflated 72%)
